In [1]:
from Bio.Seq import Seq
import itertools
import collections
import array
import numpy as np
import numpy

In [4]:
# Parsiranje referentnog genome iz .fasta formata
def readFASTA(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignorisi zaglavlje
            if not line[0] == '>':
                genome += line.rstrip()
    return genome


# Parsiranje read-ova i ocena kvaliteta FASTQ fajla
def readFASTQ(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # preskoci naziv
            seq = fh.readline().rstrip()  # procitaj sekvencu baza
            fh.readline()  # preskoci liniju +
            qual = fh.readline().rstrip() # procitaj sekvencu ocena kvaliteta
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

def phred33ToQ(qual):
    return ord(qual) - 33

# vraca reverse complement
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

seqs, quals = readFASTQ('test.fastq')

In [5]:
ref = readFASTA('MT.fa')
reverseCompl_ref = reverseComplement(ref)

In [6]:
#print(ref)

In [7]:
#print(seqs)

In [8]:
reverseSeqs = [] #lista reverse complement sekvenci read-ova test.fastq fajla
for i in range(len(seqs)):
    reverseSeqs.append(reverseComplement(seqs[i]))

In [9]:
#print(reverseSeqs)

In [10]:
def SeedP(l, d):   #l - duzina seed-a, d -> duzina rastojanja izmedju seed-ova
    seeds = []
    for j in range(len(seqs)):
        s = []
        for i in range(0,len(seqs[j]),d):
            s.append(seqs[j][i:(i+l)])
        seeds.append(s)
    return seeds

seeds = SeedP(20,10)

In [11]:
#print(seeds)

In [12]:
#Reverse complement seed-ova
def SeedRC(l, d):   #l - duzina seed-a, d -> interval izmedju seed-ova
    ReverseCSeeds = []
    for j in range(len(reverseSeqs)):
        rs = []
        for i in range(0,len(reverseSeqs[j]),d):
            rs.append(reverseSeqs[j][i:(i+l)])
        ReverseCSeeds.append(rs)
    return ReverseCSeeds

ReverseSeeds = SeedRC(20,10)

In [13]:
#print(ReverseSeeds)

In [14]:
# Exact matching - naive algorithm


# Vraca listu pojavljivanja (offset od pocetka reference)
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  
        match = True
        for j in range(len(p)):  
            if t[i+j] != p[j]:  
                match = False
                break
        if match:
            occurrences.append(i) 
    return occurrences

# naive exact matching algorithm that is strand-aware
#Osim sto trazi sve pojave P u T, trazi i pojavu reverse complement-a od P u T 
def naiveRC(p, t):
    r = reverseComplement(p)
    if r == p:
        return naive(p,t)
    else:
        return naive(p,t) + naive(r,t)


In [15]:
#Exact matching - Boyer Moore

import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGTN'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [16]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [17]:
# Seed-ovi i pozicije njihovih exact match-eva u referenci koriscenjem naivnog algoritma

def pojava(s):
    p = {}
    for i in range(len(s)):
        for j in range(len(s[i])):
            p[s[i][j]] = naive(s[i][j], ref)
    return p        

pojavljivanje = pojava(seeds)

In [18]:
#print(pojavljivanje)

In [19]:
# Reverse complement seed-ova i pozicije njihovih exact match-eva u referenci koriscenjem naivnog algoritma

def pojavaRC(rs):
    RCP = {}
    for i in range(len(rs)):
        for j in range(len(rs[i])):
            RCP[rs[i][j]] = naive(rs[i][j], ref)
    return RCP

ReverseCPojavljivanje = pojavaRC(ReverseSeeds)

In [20]:
#print(ReverseCPojavljivanje)

In [21]:
# Seed-ovi i pozicije njihovih exact match-eva u referenci koriscenjem Boyer-Moore-ovog algoritma

def pojavaBM(s):
    pBM = {}
    for i in range(len(s)):
        for j in range(len(s[i])):
            p = s[i][j]
            p_bm = BoyerMoore(p, alphabet='ACGTN')
            pBM[p] = boyer_moore(p, p_bm, ref)
    return(pBM)

In [22]:
# Reverse complement seed-ova i pozicije njihovih exact match-eva u referenci koriscenjem Boyer-Moore-ovog algoritma

def pojavaRCBM(rs):
    RCPBM = {}
    for i in range(len(rs)):
        for j in range(len(rs[i])):
            p = rs[i][j]
            p_bm = BoyerMoore(p, alphabet='ACGTN')
            RCPBM[p] = boyer_moore(p, p_bm, ref)
    return(RCPBM) 

In [24]:
def remove_empty_keys(d):
    l = []
    for k in list(d):
        if not d[k]:
            del d[k]

            
v = dict(pojavljivanje)
remove_empty_keys(v)
vr = v.values()

flattened_vr = [y for x in vr for y in x]

#referentni genom se deli na round(max(flattened_vr)/450) prozora cije tezine su smestene u nizu a
#prozor dobija vecu tezinu ako postoji veci broj seed-ova koji su se match-ovali na referencu u tom prozoru
a = array.array('i',(0 for i in range(0, round(max(flattened_vr)/450))))

def pod(niz):
    for i in range(len(flattened_vr)):
        for j in range(0, round(max(flattened_vr)/450)):
            if (flattened_vr[i] > j*450 and flattened_vr[i] < ((j+1)*450-1)):
                niz[j] = niz[j] + 1
    return(niz)

nn = pod(a)
#print(nn)

In [25]:
arr = np.array(a)
brr = arr.argsort()[-10:][::-1]

#lista odabranih prozora u referenci sortiranih po tezini
ref_iz = []
for i in range(len(brr)):
    ref_iz.append(ref[brr[i]*450:(brr[i]+1)*450])

In [26]:
#print(ref_iz)

In [28]:
vRC = dict(ReverseCPojavljivanje)
remove_empty_keys(vRC)
vrRC = vRC.values()
#print(vrRC)

flattened_vrRC = [y for x in vrRC for y in x]
#print(flattened_vrRC)

#referentni genom se deli na round(max(flattened_vr)/450) prozora cije tezine su smestene u nizu aRC
#prozor dobija vecu tezinu ako postoji veci broj reverse complement seed-ova koji su se match-ovali na referencu u tom prozoru
aRC = array.array('i',(0 for i in range(0, round(max(flattened_vrRC)/450))))

def podRC(nizRC):
    for i in range(len(flattened_vrRC)):
        for j in range(0, round(max(flattened_vrRC)/450)):
            if (flattened_vrRC[i] > j*450 and flattened_vrRC[i] < ((j+1)*450-1)):
                nizRC[j] = nizRC[j] + 1
    return(nizRC)

nnRC = podRC(aRC)
#print(nnRC)

In [29]:
arrRC = np.array(aRC)
brrRC = arrRC.argsort()[-10:][::-1]

#lista odabranih prozora u referenci sortiranih po tezini
ref_izRC = []
for i in range(len(brrRC)):
    ref_izRC.append(ref[brrRC[i]*450:(brrRC[i]+1)*450])

In [30]:
#print(ref_izRC)

In [31]:
#Local Alignment

def scoringMatrix(a,b):
    if a==b: return 1 # match
    if a=='_' or b=='_': return -7 # indel
    return -4 # mismatch

In [32]:
def localAlignment(x,y, s):
    D = numpy.zeros((len(x)+1,len(y)+1), dtype=int)

    for i in range(1,len(x)+1):
        for j in range(1,len(y)+1):
            D[i,j]=max(0, 
                       D[i-1,j]  +s(x[i-1], '_'),
                       D[i,j-1]  +s('_',   y[j-1]), 
                       D[i-1,j-1]+s(x[i-1],y[j-1]))
    
    # find the cell in the table which has maximum value       
    localMax = D.max()
    return D, localMax

In [37]:
x=ref_iz[0]
y = seqs[5]
V, score = localAlignment(x, y, scoringMatrix)
#print(V)

In [38]:
#print("Best score is: %d in cell %s" % (score, numpy.unravel_index(numpy.argmax(V), V.shape)))

In [39]:
def tracebak(x,y,V, s):
    maxValue=numpy.where(V==V.max())
    i=maxValue[0][0]
    j=maxValue[1][0]
    ax,ay,am, tr = '','','',''
    while (i>0 or j>0) and V[i,j]!=0:
        if i>0 and j>0:
            sigma = 1 if x[i-1]==y[j-1] else 0
            d=V[i-1,j-1]+s(x[i-1],y[j-1]) # diagonal move
        if i>0: v=V[i-1,j]+s(x[i-1],'_')  # vertical move
        if j>0: h=V[i,j-1]+s('_',y[j-1])  # horizontal move
        
        # diagonal is the best
        if d>=v and d>=h:
            ax += x[i-1]
            ay += y[j-1]
            if sigma==1:
                tr+='M'
                am+='|'
            else:
                tr+='R'
                am+=' '
            i-=1
            j-=1
        # vertical is the best
        elif v>=h:
            ax+=x[i-1]
            ay+='_'
            tr+='I'
            am+=' '
            i-=1
        # horizontal is the best
        else:
            ay+=y[j-1]
            ax+='_'
            tr+='D'
            am+=' '
            j-=1
    alignment= '\n'.join([ax[::-1], am[::-1], ay[::-1]])
    return alignment, tr[::-1]

In [40]:
def SAMoutput(self, outf, read, pos, max, t):
        with open(outf) as file:
            file.write("{}  {}  {}  {}  {}  {}  {}  {}\n".format(
                read[0], "MT", pos, max, t, read[1], read[2]))